# Import library and Load data

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np

file_path = "../data/train.csv"
dataset = pd.read_csv(file_path)
print("Datasets shape {}".format(dataset.shape))

# display data entries
print(dataset.head(10))

Datasets shape (1460, 81)
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN      Reg   

  LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1        

# Feature selection

In [2]:
#Continuous features
kitchen_quality_column = 'KitchenQual'
continuous_features = ['LotArea', 'YearBuilt', '1stFlrSF', 'GrLivArea']

#categorical features
kitchen_quality_column = 'KitchenQual'
categorical_features = ['Neighborhood', 'HouseStyle', 'OverallQual', 'OverallCond',kitchen_quality_column]

#Targeted value
targeted = 'SalePrice'

#check for one hot or cardinal encoding
features_for_one_hot_encoding = ['Neighborhood', 'HouseStyle', 'OverallQual', 'OverallCond']
features_for_ordinal_encoding = [kitchen_quality_column]

# Total features extracted from dataset
features = dataset[continuous_features + categorical_features]
target = dataset [targeted] 
print(features, target)

      LotArea  YearBuilt  1stFlrSF  GrLivArea Neighborhood HouseStyle  \
0        8450       2003       856       1710      CollgCr     2Story   
1        9600       1976      1262       1262      Veenker     1Story   
2       11250       2001       920       1786      CollgCr     2Story   
3        9550       1915       961       1717      Crawfor     2Story   
4       14260       2000      1145       2198      NoRidge     2Story   
...       ...        ...       ...        ...          ...        ...   
1455     7917       1999       953       1647      Gilbert     2Story   
1456    13175       1978      2073       2073       NWAmes     1Story   
1457     9042       1941      1188       2340      Crawfor     2Story   
1458     9717       1950      1078       1078        NAmes     1Story   
1459     9937       1965      1256       1256      Edwards     1Story   

      OverallQual  OverallCond KitchenQual  
0               7            5          Gd  
1               6            8   

# Feature processing

In [39]:
# Initializing the scalers
scaler = StandardScaler()

# Applying One hot encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(features[features_for_one_hot_encoding])
one_hot_encoder = encoder.transform(features[features_for_one_hot_encoding])


# Convert numpy arrays to DataFrames
one_hot_encoded_df = pd.DataFrame(one_hot_encoder, columns=encoder.get_feature_names_out(['Neighborhood', 'HouseStyle', 'OverallQual', 'OverallCond']))

# for cardinal encoder
kitchen_quality_dict = {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1}
kitchen_quality_encoded = features[kitchen_quality_column].apply(lambda x: kitchen_quality_dict[x])
kitchen_quality_encoded.head()

kitchen_quality_encoded.apply(lambda x: x * 2).head(15)
features[kitchen_quality_column].apply(lambda x: x.lower()).head(15)

# Scaling
scaler.fit(features[continuous_features])
scaled_data = scaler.transform(features[continuous_features])

# Combine the preprocessed features
scaled_df = pd.DataFrame(data = scaled_data, columns = continuous_features)

X = pd.concat([one_hot_encoded_df, kitchen_quality_encoded, scaled_df], axis=1)


# Model training

In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Model evaluation with the competition metric

In [28]:
# Define the function for Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value   
#and the logarithm of the observed sales price
def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_error(y_test, y_pred))
    return round(rmsle, precision)

# Calculate the RMSLE for training and testing sets
rmsle_train = compute_rmsle(y_train, y_pred_train)
rmsle_test = compute_rmsle(y_test, y_pred_test)

print(f'Training RMSLE: {rmsle_train}')
print(f'Testing RMSLE: {rmsle_test}')

Training RMSLE: 31469.27
Testing RMSLE: 30707.51


In [29]:
X_train

Neighborhood_Blueste  Neighborhood_BrDale  Neighborhood_BrkSide  \
1023                   0.0                  0.0                   0.0   
810                    0.0                  0.0                   0.0   
1384                   0.0                  0.0                   0.0   
626                    0.0                  0.0                   0.0   
813                    0.0                  0.0                   0.0   
...                    ...                  ...                   ...   
1095                   0.0                  0.0                   0.0   
1130                   0.0                  0.0                   0.0   
1294                   0.0                  0.0                   0.0   
860                    0.0                  0.0                   0.0   
1126                   0.0                  0.0                   0.0   

      Neighborhood_ClearCr  Neighborhood_CollgCr  Neighborhood_Crawfor  \
1023                   0.0                   0.0                   0.0   
810                    0.0                   0.0                   0.0   
1384                   0.0                   0.0                   0.0   
626                    0.0                   0.0                   0.0   
813                    0.0                   0.0                   0.0   
...                    ...                   ...                   ...   
1095                   0.0                   1.0                   0.0   
1130                   0.0                   0.0                   0.0   
1294                   0.0                   0.0                   0.0   
860                    0.0                   0.0                   1.0   
1126                   0.0                   0.0                   0.0   

      Neighborhood_Edwards  Neighborhood_Gilbert  Neighborhood_IDOTRR  \
1023                   0.0                   0.0                  0.0   
810                    0.0                   0.0                  0.0   
1384                   1.0                   0.0                  0.0   
626                    0.0                   0.0                  0.0   
813                    0.0                   0.0                  0.0   
...                    ...                   ...                  ...   
1095                   0.0                   0.0                  0.0   
1130                   0.0                   0.0                  0.0   
1294                   1.0                   0.0                  0.0   
860                    0.0                   0.0                  0.0   
1126                   0.0                   0.0                  0.0   

      Neighborhood_MeadowV  ...  OverallCond_5  OverallCond_6  OverallCond_7  \
1023                   0.0  ...            1.0            0.0            0.0   
810                    0.0  ...            0.0            1.0            0.0   
1384                   0.0  ...            1.0            0.0            0.0   
626                    0.0  ...            1.0            0.0            0.0   
813                    0.0  ...            0.0            1.0            0.0   
...                    ...  ...            ...            ...            ...   
1095                   0.0  ...            1.0            0.0            0.0   
1130                   0.0  ...            0.0            0.0            0.0   
1294                   0.0  ...            0.0            0.0            1.0   
860                    0.0  ...            0.0            0.0            0.0   
1126                   0.0  ...            1.0            0.0            0.0   

      OverallCond_8  OverallCond_9  KitchenQual   LotArea  YearBuilt  \
1023            0.0            0.0            3 -0.735111   1.117235   
810             0.0            0.0            3 -0.037766   0.090492   
1384            0.0            0.0            2 -0.146006  -1.068734   
626             0.0            0.0            2  0.182922  -0.373198   
813             0.0            0.0        

In [ ]:
X_test

In [30]:
y_pred = model.predict(X_test)
y_pred[:5]

array([146518.94931781, 329489.02310858, 105673.40712581, 168191.39365358,
       323842.17924014])

In [31]:
y_pred

array([146518.94931781, 329489.02310858, 105673.40712581, 168191.39365358,
       323842.17924014,  87075.34647248, 247645.9409465 , 137485.96972361,
        86355.777178  , 132222.45556876, 145925.253611  , 127552.70998524,
        82142.77622115, 238526.32271373, 174747.25751783, 140890.54619203,
       201040.15944041, 139830.46410478, 100020.67660571, 205357.02331127,
       156087.78327287, 238834.74925763, 177131.86533206, 126101.83468873,
       199669.3409697 , 165430.73888241, 197351.4411345 , 119003.13974819,
       169248.25401443, 206227.85581371, 115092.56723641, 253532.1098362 ,
       295018.5800515 , 127774.73404964, 259893.94012198, 147528.49811519,
       149898.93888892, 216193.19042803, 342889.45044507, 102008.52706626,
       125699.00210138, 211973.2263634 , 119262.12041776, 312811.61654373,
       113834.48599487, 118513.45823822, 112446.59752521, 130614.68109354,
       376614.56805482, 136701.93687131, 126839.66860083, 177681.5878008 ,
       108441.43795114, 3

In [42]:
X.to_parquet('../data/processed_df.parquet', index=False, engine='pyarrow')

In [43]:
expected_processed_df = pd.read_parquet('../data/processed_df.parquet')

In [46]:
pd.testing.assert_frame_equal(X, expected_processed_df)